In [1]:
import time
import pandas as pd
from utils import mysqlDatabase, dcardApi

In [2]:
class postCrawler:
    def __init__(self, database_username, database_password, database_ip, database_name, base_url, popular, max_limit):
        self.database_username = database_username
        self.database_password = database_password
        self.database_ip = database_ip
        self.database_name = database_name
        self.base_url = base_url
        self.popular = popular
        self.max_limit = max_limit
    def get_forums_list(self, MysqlDatabase, sql):
        df = MysqlDatabase.select_table(sql)
        forums_list = df['alias'].tolist()
        return forums_list
    def get_new_post(self, DcardApi, forums_alias):
        df = DcardApi.get_Dcard_posts(forums_alias)
        return df
    def get_old_post(self, DcardApi, forums_alias, before_postid):
        df = DcardApi.get_Dcard_posts(forums_alias, before_postid)
        return df
    def get_forums_before_postid(self, MysqlDatabase, forums_alias):
        sql = '''
        SELECT df.name, df.alias, dp.id, dp.title, dp.createdAT
        FROM Bigdata.dcard_posts dp
        left join Bigdata.dcard_forums df on dp.forumid = df.id
        where 1=1
        and df.alias = ':forums_alias'
        order by createdAT desc
        '''
        sql = sql.replace(':forums_alias', forums_alias)
        df = MysqlDatabase.select_table(sql)
        try:
            before_postid = df.head(1)['id'].tolist()[0]
        except:
            before_postid = None
        return before_postid
    def main(self):
        '''
        實作
        '''
        MysqlDatabase = mysqlDatabase(self.database_username, self.database_password, self.database_ip, self.database_name)
        DcardApi = dcardApi(self.base_url, self.popular, self.max_limit)
        ## 看板列表
        my_interest_forums = ['時事', '網路購物', '股票', '美妝', '工作', '考試', '穿搭', '3C', 'Apple', '感情', 
                              '美食', '理財', '居家生活', '臺灣大學', 'YouTuber']
        sql = '''
        select *
        from Bigdata.dcard_forums
        where 1=1
        and name in :my_interest_forums
        '''
        sql = sql.replace(':my_interest_forums', str(tuple(my_interest_forums)))
        forums_list = self.get_forums_list(MysqlDatabase, sql)

        ## 最新文章
        df_post = pd.DataFrame(columns=['id', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'author', 'gender', 'createdAt'])
        time_start = time.time() 
        for alias in forums_list:
            time.sleep(1)
            time_cost = time.time() - time_start
            if time_cost >= 3600:
                break
            try:
                print(alias)
                df_post = pd.concat([df_post, self.get_new_post(DcardApi, alias)])
                before_postid = PostCrawler.get_forums_before_postid(MysqlDatabase, alias)
                df_post = pd.concat([df_post, self.get_old_post(DcardApi, alias, before_postid)])
            except:
                print('connection abort, need some break')
                time.sleep(5)
                continue
        print('total time cost(sec): ', time_cost)
        df_post = df_post.fillna('')[['id', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'author', 'gender', 'createdAt']]
        MysqlDatabase.upsert_table(df_post, table_name='dcard_posts')
        return df_post

In [3]:
if __name__ == '__main__':
    database_username = 'jackyfu'
    database_password = 'data8756'
    database_ip       = 'sg2nlmysql29plsk.secureserver.net'
    database_name     = 'Bigdata'
    base_url = 'https://www.dcard.tw/service/api/v2'
    popular = 'false'
    max_limit = '100'
    ###
    PostCrawler = postCrawler(database_username, database_password, database_ip, database_name, base_url, popular, max_limit)
    df_post = PostCrawler.main()

Successfully select from Bigdata table
ntu
Successfully select from Bigdata table
total time cost(sec):  1.0050239562988281
